<a href="https://colab.research.google.com/github/Kalima83/procesamiento_lenguaje_natural_Desafios/blob/main/desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de lenguaje con tokenización por caracteres
## Consigna

1. Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.

2. Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.

3. Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.

4. Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

## Sugerencias
Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.

Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU. rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

In [ ]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

## Datos
Utilizaremos como dataset un libro en castellano "El Conde de Montecristo" de
Alejandro Dumas.

In [4]:
# descargar de textos.info
import urllib.request

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

In [5]:
raw_html = urllib.request.urlopen('https://www.textos.info/alejandro-dumas/el-conde-de-montecristo/ebook')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag )
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

# pasar todo el texto a minúscula
article_text = article_text.lower()

In [6]:
# en article text se encuentra el texto de todo el libro
article_text[:1000]

' el 24 de febrero de 1815, el vigía de nuestra señora de la guarda dio\n la señal de que se hallaba a la vista el bergantín el faraón procedente\n de esmirna, trieste y nápoles. como suele hacerse en tales casos, salió\n inmediatamente en su busca un práctico, que pasó por delante del \ncastillo de if y subió a bordo del buque entre la isla de rión y el cabo\n mongión. en un instante, y también como de costum\xadbre, se llenó de \ncuriosos la plataforma del castillo de san juan, por\xadque en marsella se \ndaba gran importancia a la llegada de un buque y sobre todo si le \nsucedía lo que al faraón, cuyo casco había salido de los astilleros de \nla antigua focia y pertenecía a un naviero de la ciudad. mientras tanto, el buque seguía avanzando; habiendo pasado \nfeliz\xadmente el estrecho producido por alguna erupción volcánica entre \nlas islas de calasapeigne y de jaros, dobló la punta de pomegue \nhendien\xaddo las olas bajo sus tres gavias, su gran foque y la mesana. lo \nhacía con 

In [7]:
# seleccionamos el tamaño de contexto
max_context_size = 100


# Usaremos las utilidades de procesamiento de textos y secuencias de Keras
from tensorflow.keras.utils import pad_sequences # se utilizará para padding


# en este caso el vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(article_text)


# la longitud de vocabulario de caracteres es:
print("La longitud de vocabulario de caracteres es: ")
len(chars_vocab)


La longitud de vocabulario de caracteres es: 


83

In [8]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

# Tokenizar

In [9]:
# tokenizamos el texto completo
# convierte todo el texto del libro en una secuencia de números,
# usando el diccionario char2idx
tokenized_text = [char2idx[ch] for ch in article_text]

print("Primeros 1000 caracteres del libro convertidos en índices numéricos")
print(tokenized_text[:1000])

Primeros 1000 caracteres del libro convertidos en índices numéricos
[21, 67, 52, 21, 23, 25, 21, 80, 67, 21, 77, 67, 29, 22, 67, 22, 69, 21, 80, 67, 21, 56, 27, 56, 18, 1, 21, 67, 52, 21, 60, 37, 39, 74, 76, 21, 80, 67, 21, 24, 12, 67, 6, 4, 22, 76, 21, 6, 67, 34, 69, 22, 76, 21, 80, 67, 21, 52, 76, 21, 39, 12, 76, 22, 80, 76, 21, 80, 37, 69, 7, 21, 52, 76, 21, 6, 67, 34, 76, 52, 21, 80, 67, 21, 30, 12, 67, 21, 6, 67, 21, 71, 76, 52, 52, 76, 29, 76, 21, 76, 21, 52, 76, 21, 60, 37, 6, 4, 76, 21, 67, 52, 21, 29, 67, 22, 39, 76, 24, 4, 74, 24, 21, 67, 52, 21, 77, 76, 22, 76, 32, 24, 21, 40, 22, 69, 31, 67, 80, 67, 24, 4, 67, 7, 21, 80, 67, 21, 67, 6, 43, 37, 22, 24, 76, 1, 21, 4, 22, 37, 67, 6, 4, 67, 21, 3, 21, 24, 50, 40, 69, 52, 67, 6, 53, 21, 31, 69, 43, 69, 21, 6, 12, 67, 52, 67, 21, 71, 76, 31, 67, 22, 6, 67, 21, 67, 24, 21, 4, 76, 52, 67, 6, 21, 31, 76, 6, 69, 6, 1, 21, 6, 76, 52, 37, 32, 7, 21, 37, 24, 43, 67, 80, 37, 76, 4, 76, 43, 67, 24, 4, 67, 21, 67, 24, 21, 6, 12, 21, 29, 12

# Organizando y estructurando el dataset

In [10]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

In [11]:
# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

In [12]:
# Train con ventana deslizante (sliding window)
tokenized_sentences_train = [
    train_text[init : init + max_context_size]
    for init in range(len(train_text) - max_context_size + 1)
]

In [13]:
# Val con segmentación
tokenized_sentences_val = [
    val_text[i*max_context_size : (i+1)*max_context_size]
    for i in range(num_val)
]

In [14]:
# Generamos los arrays
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

X_val = np.array(tokenized_sentences_val[:-1])
y_val = np.array(tokenized_sentences_val[1:])

In [15]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,21,67,52,21,23,25,21,80,67,21,...,21,71,76,52,52,76,29,76,21,76
1,67,52,21,23,25,21,80,67,21,77,...,71,76,52,52,76,29,76,21,76,21
2,52,21,23,25,21,80,67,21,77,67,...,76,52,52,76,29,76,21,76,21,52
3,21,23,25,21,80,67,21,77,67,29,...,52,52,76,29,76,21,76,21,52,76
4,23,25,21,80,67,21,77,67,29,22,...,52,76,29,76,21,76,21,52,76,21


In [16]:
pd.DataFrame(y).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,67,52,21,23,25,21,80,67,21,77,...,71,76,52,52,76,29,76,21,76,21
1,52,21,23,25,21,80,67,21,77,67,...,76,52,52,76,29,76,21,76,21,52
2,21,23,25,21,80,67,21,77,67,29,...,52,52,76,29,76,21,76,21,52,76
3,23,25,21,80,67,21,77,67,29,22,...,52,76,29,76,21,76,21,52,76,21
4,25,21,80,67,21,77,67,29,22,67,...,76,29,76,21,76,21,52,76,21,60


In [17]:
X.shape

(2294651, 100)

In [18]:
vocab_size = len(chars_vocab)

# Definiendo el modelo
Utilizo los modelos recomendados

In [19]:
import numpy as np
from tensorflow.keras.callbacks import Callback

#Callback
class PerplexityCallback(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_ppl = np.inf

    def on_epoch_end(self, epoch, logs=None):
        preds = self.model.predict(self.X_val, verbose=0)
        # evitar log(0)
        preds = np.clip(preds, 1e-10, 1.0)
        # one-hot implícito con sparse targets
        probs = preds[np.arange(len(self.y_val)), self.y_val.flatten()]
        cross_entropy = -np.mean(np.log(probs))
        ppl = np.exp(cross_entropy)

        print(f"\n>>> Perplexity val: {ppl:.3f}")

        # early stopping manual
        if ppl < self.best_ppl:
            self.best_ppl = ppl
            print("Mejoró. Guardando pesos.")
            self.model.save_weights("best_weights.h5")
        else:
            print("No mejoró. Deteniendo entrenamiento.")
            self.model.stop_training = True


In [20]:
perplexity_callback = PerplexityCallback(X_val, y_val)

## SimpleRNN

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers import RMSprop

embed_dim = 64
rnn_units = 128

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_context_size),
    SimpleRNN(rnn_units, return_sequences=True),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(0.001)
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## LSTM

In [22]:
from tensorflow.keras.layers import LSTM

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_context_size),
    LSTM(rnn_units, return_sequences=True),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(0.001)
)

## GRU

In [23]:
from tensorflow.keras.layers import GRU

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_context_size),
    GRU(rnn_units, return_sequences=True),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(0.001)
)

## Entrenar usando el callback

In [ ]:
history = model.fit(
    X, y,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    callbacks=[perplexity_callback]
)

In [ ]:
#Cargar mejores pesos
model.load_weights("best_weights.h5")

# Funciones de generación

## Greedy Search

In [ ]:
def generate_greedy(model, seed, length=300):
    seq = [char2idx[c] for c in seed]
    for _ in range(length):
        x = np.array(seq[-max_context_size:])[None, :]
        preds = model.predict(x, verbose=0)[0][-1]
        next_char = np.argmax(preds)
        seq.append(next_char)
    return "".join(idx2char[i] for i in seq)

## Sampling con temperatura

In [ ]:
def sample_with_temperature(preds, T=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-9) / T
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_with_temperature(model, seed, T=1.0, length=300):
    seq = [char2idx[c] for c in seed]
    for _ in range(length):
        x = np.array(seq[-max_context_size:])[None, :]
        preds = model.predict(x, verbose=0)[0][-1]
        next_char = sample_with_temperature(preds, T)
        seq.append(next_char)
    return "".join(idx2char[i] for i in seq)

## Beam Search determinístico

In [ ]:
def beam_search(model, seed, k=3, length=200):
    seq = [char2idx[c] for c in seed]
    beams = [(seq, 0)]

    for _ in range(length):
        new_beams = []
        for seq, score in beams:
            x = np.array(seq[-max_context_size:])[None, :]
            preds = model.predict(x, verbose=0)[0][-1]
            top_k = np.argsort(preds)[-k:]

            for token in top_k:
                new_seq = seq + [token]
                new_score = score + np.log(preds[token] + 1e-9)
                new_beams.append((new_seq, new_score))

        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:k]

    best_seq = beams[0][0]
    return "".join(idx2char[i] for i in best_seq)

In [ ]:
#Comprobación
print("===== GREEDY =====")
print(generate_greedy(model, seed, 300))

print("\n===== BEAM SEARCH (k=3) =====")
print(beam_search(model, seed, beam_width=3, length=300))

print("\n===== TEMPERATURE 0.7 =====")
print(sample_with_temperature(model, seed, length=300, T=0.7))

print("\n===== TEMPERATURE 1.2 =====")
print(sample_with_temperature(model, seed, length=300, T=1.2))
